In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
from flask import Flask, render_template, Response


In [2]:
app = Flask(__name__)

In [3]:
# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose
 
# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=2)
 
# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils

# Flag to track if yoga session has started.
yoga_started = False

In [4]:
def detectPose(image, pose):    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    return output_image, landmarks

In [5]:
# Setup Pose function for video.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

In [6]:
# Function to display text on image.
def displayText(image, text, position, font=cv2.FONT_HERSHEY_PLAIN, font_scale=2, color=(0, 255, 255), thickness=2):
    cv2.putText(image, text, position, font, font_scale, color, thickness)

# Function to display countdown timer.
def displayTimer(image, seconds_left, position=(400, 50), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=2, color=(0, 255, 0), thickness=2,box_color=(255,0,0)):
    # Draw red box
    text_size, _ = cv2.getTextSize(str(seconds_left), font, font_scale, thickness)
    box_padding = 10  # Adjust the padding as needed
    box_coords = ((position[0] - box_padding, position[1] - text_size[1] - box_padding), 
                  (position[0] + text_size[0] + box_padding, position[1] + box_padding))
    cv2.rectangle(image, box_coords[0], box_coords[1], box_color, thickness=cv2.FILLED)

    # Display timer text
    cv2.putText(image, str(seconds_left), position, font, font_scale, color, thickness)

In [7]:
def calculateAngle(landmark1, landmark2, landmark3):

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [8]:
def classifyPose(landmarks, output_image):

    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    # angle between the left pinky, wrist and elbow points
    left_wrist_angle=calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
    
    # angle between the right pinky, wrist and elbow points
    right_wrist_angle=calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    
    left_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
    
    right_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
     
    # # #----------------------------------------------------------------------------------------------------------------
    
    # # Virabhadrasana pose or the T pose.
    # #----------------------------------------------------------------------------------------------------------------
    
    # # Check if the both arms are straight.
    # if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:

    #     # Check if shoulders are at the required angle.
    if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:

    # Check if it is the Virabhadrasana.
    #----------------------------------------------------------------------------------------------------------------

        # Check if one leg is straight.
        if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

            # Check if the other leg is bended at the required angle.
            if left_knee_angle > 90 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:

                # Specify the label of the pose that is Warrior II pose.
                label = 'Virabhadrasana' 
                        
    # #----------------------------------------------------------------------------------------------------------------        
    #----------------------------------------------------------------------------------------------------------------
    # SuryaNamastar - Step 1:
    #Pranamasana
    #----------------------------------------------------------------------------------------------------------------

    if 20< left_shoulder_angle < 50 and 20 < right_shoulder_angle < 50:
        if 280<left_elbow_angle < 360 and 30<right_elbow_angle < 60:
            if 140<left_wrist_angle < 200 and 170<right_wrist_angle < 225:
                label = 'Pranamasana'
                
    #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 2:
    # HashaUtanasana
    #----------------------------------------------------------------------------------------------------------------
    
    if 160< left_shoulder_angle < 180 and 160 < right_shoulder_angle < 180:
        if 160<left_elbow_angle < 195 and 160<right_elbow_angle < 195:
            label = 'Hastauttanasana'
    
     #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 3:
    # Uttanasana
    #----------------------------------------------------------------------------------------------------------------
    
    if 155<left_hip_angle<200 and 180<right_hip_angle<210:
        if 165<left_elbow_angle<190 and 180<right_elbow_angle<200:
            label = 'Uttanasana'   
             
    #  #----------------------------------------------------------------------------------------------------------------           
    # # SuryaNamastar - Step 4:
    # # Ashwa Sanchalanasana 
    # #----------------------------------------------------------------------------------------------------------------
    
    if 160<right_knee_angle<190:
        if 170<left_elbow_angle<195 and 170<right_elbow_angle<195:
            label='Sanchalanasana(R)'  
     
    #----------------------------------------------------------------------------------------------------------------                   
    # SuryaNamastar -Step 5:
    # Chuturanga Dandasana
    # ----------------------------------------------------------------------------------------------------------------

    if 170<left_elbow_angle<195 and 170<right_elbow_angle<195:
        if 160<left_hip_angle<180 and 175<right_hip_angle<195:
            label= 'Chaturanga Dandasana'
    
     #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 7:
    # BHUJANGASANA
    #----------------------------------------------------------------------------------------------------------------
    
    if 170<left_elbow_angle<190 and 175<right_elbow_angle<210:
        if 135<left_hip_angle<155 and 145<right_hip_angle<165:
            label= 'BHUJANGASANA'
    
    #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 8:
    # ADHO MUKHA SAVASANA
    #----------------------------------------------------------------------------------------------------------------
    
    if 170<left_elbow_angle<195 and 170<right_elbow_angle<195:
        if 10<left_hip_angle<30 and 300<right_hip_angle<350:
            label= 'ADHO MUKHA SAVASANA'
    
    #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 9:
    # Ashwa Sanchalanasana (Left)
    #----------------------------------------------------------------------------------------------------------------
   
    if 160<left_knee_angle<190:
        if 170<left_elbow_angle<195 and 170<right_elbow_angle<195:
            label='Sanchalanasana(L)'
     
    #----------------------------------------------------------------------------------------------------------------           
    # SuryaNamastar - Step 10:
    # Uttanasana
    #----------------------------------------------------------------------------------------------------------------
   
    if 155<left_hip_angle<200 and 180<right_hip_angle<210:
        if 165<left_elbow_angle<190 and 180<right_elbow_angle<200:
            label = 'Uttanasana'    
        
    
    # SuryaNamastar - Step 11:
    # HashaUtanasana
    #----------------------------------------------------------------------------------------------------------------

    if 160< left_shoulder_angle < 180 and 160 < right_shoulder_angle < 180:
        if 160<left_elbow_angle < 195 and 160<right_elbow_angle < 195:
            label = 'Hastauttanasana'
    
    #----------------------------------------------------------------------------------------------------------------
    # SuryaNamastar - Step 12:
    #Pranamasana
    #----------------------------------------------------------------------------------------------------------------
    
    if 20< left_shoulder_angle < 50 and 20 < right_shoulder_angle < 50:
        if 280<left_elbow_angle < 360 and 30<right_elbow_angle < 60:
            # Check if wrists are close to the body (optional condition).
            if 140<left_wrist_angle < 200 and 170<right_wrist_angle < 225:
                # Specify the label of the pose that is Pranamasana.
                label = 'Pranamasana'   
        
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

    # Return the output image and the classified label.
    return output_image, label

In [9]:
##Main Function

# Load the background image
background_image = cv2.imread('Yoga.jpg')

if background_image is None:
    print("Error: Unable to load the background image.")
else:
    # Define the frame size
    frame_height, frame_width = 480, 640

    # Create a blank frame with the same size as the background image
    blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Resize the background image to match the frame size
    background_image = cv2.resize(background_image, (frame_width, frame_height))

    # Overlay the background image on the blank frame
    overlayed_frame = cv2.addWeighted(blank_frame, 0.5, background_image, 0.5, 0)
    
    # Flag to track if yoga session has started.
    yoga_started = False

    # Start time for countdown.
    start_time = None
    
    # Initialize the VideoCapture object to read from the webcam.
    camera_video = cv2.VideoCapture(0)

    # # Initialize the VideoCapture object to read from a video stored in the disk.
    # camera_video = cv2.VideoCapture('Videp.mp4')

    cv2.namedWindow('Pose Classification', cv2.WINDOW_NORMAL)


    # Iterate until the webcam is accessed successfully.
    while camera_video.isOpened():
        
        # Read a frame.
        ok, frame = camera_video.read()
        
        # Check if frame is not read properly.
        if not ok:
            
            break
        
        # Flip the frame horizontally for natural (selfie-view) visualization.
        frame = cv2.flip(frame, 1)
        
        # Resize the frame while keeping the aspect ratio.
        frame = cv2.resize(frame, (500, 640))
        
        
        if not yoga_started:
            displayText(overlayed_frame, "Are you ready to do YOGA!", (100, 250), font=cv2.FONT_HERSHEY_COMPLEX, font_scale=1, color=(0, 255, 255), thickness=2)
            cv2.imshow('Pose Classification', overlayed_frame)
            cv2.waitKey(5000)  # Wait for 5 seconds
            yoga_started = True
            
         # Display "Enter 'F' to start" only when yoga session hasn't started yet.
        if not yoga_started:
            displayText(frame, "Enter 'F' to start", (50, 300), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, color=(255, 255, 0), thickness=2)
        
        # Wait for 'F' key press to start the session.
        key = cv2.waitKey(1) & 0xFF
        if key == ord('f'):
            yoga_started = True
            start_time = time()  # Start timer
        
        # Display countdown timer for 10 seconds after pressing 'S' key.
        if key == ord('s'):
            start_time = time()  # Reset timer
            
        if start_time is not None:
            elapsed_time = time() - start_time
            seconds_left = max(10 - int(elapsed_time), -1)
            if seconds_left >= 0:
                displayTimer(frame, seconds_left)
                
        # Perform Pose landmark detection.
        frame, landmarks = detectPose(frame, pose_video)
        
        # Check if the landmarks are detected.
        if landmarks and start_time is not None:
            
            # Perform the Pose Classification.
            frame, _ = classifyPose(landmarks, frame)
        
        # Display the frame.
        cv2.imshow('Pose Classification', frame)
        
        # Wait until a key is pressed.
        # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) & 0xFF
        
        # Check if 'ESC' is pressed.
        if(k == 27):
            # Break the loop.
            break

    # Release the VideoCapture object and close the windows.
    camera_video.release()                                                                                                      
    cv2.destroyAllWindows()


In [10]:
@app.route('/')
def index():
    return render_template('index.html')

def generate_frames():
    # Your existing code for capturing frames...

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(debug=True)

IndentationError: expected an indented block (2671033709.py, line 8)